<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/Liver_Ultrasound_Detection_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 2 Hackathon - Liver_Ultrasound_Detection_Classification

## Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## Custom Dataset For Classification

In [ ]:
import pandas as pd
from PIL import Image , ImageOps
import numpy as np
import os


In [ ]:
train_annotation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv'
validation_annotation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv'

In [ ]:
train_annotation_df = pd.read_csv(train_annotation_path)

In [ ]:
train_annotation_df.iloc[0]['file_name']

In [ ]:
len(train_annotation_df)

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets , transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

In [ ]:
class UltraSoundDataset (Dataset) :

    def __init__ (self , annotation_path , image_path , transform = None , target_transform = None) :

        self.annotation_path = annotation_path
        self.annotation_file = pd.read_csv(annotation_path)

        self.image_path = image_path

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) :

        return len(self.annotation_file)

    def __getitem__ (self , idx) :

        image_name = self.annotation_file.iloc[idx]['file_name']

        image = Image.open(f'{self.image_path}/{image_name}.jpg')

        label = self.annotation_file.iloc[idx]['class']

        if self.transform :

            image = self.transform(image)

        if self.target_transform :n

            label = self.target_transform(label)

        return image , label


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224 , 224)),  # Resize the image to 128x128
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
])

In [ ]:
target_transform = Lambda(lambda y: torch.zeros(2, dtype=torch.float).scatter_(dim = 0, index = torch.tensor(y), value = 1))

In [ ]:
annotation_for_train_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv'
train_image_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/train/images'

annotation_for_validation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv'
validation_image_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/validation/images'

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataset = UltraSoundDataset(annotation_for_train_path , train_image_path , transform , target_transform)
train_dataloader = DataLoader(train_dataset , batch_size = 16, shuffle = True , num_workers = 8)

validation_dataset = UltraSoundDataset(annotation_for_validation_path , validation_image_path , transform , target_transform)
validation_dataloader = DataLoader(validation_dataset , batch_size = 16, shuffle = False , num_workers = 8)

## Set Varibles For Model Training | MaxVit

In [ ]:
!pip install timm
!pip install evaluate

In [ ]:
import timm
import timm.optim
import timm.scheduler
from timm.data import ImageDataset, create_dataset, create_loader
from timm.data.transforms_factory import create_transform
from torch import nn

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from PIL import Image

import evaluate

import numpy as np
import pandas as pd
from scipy import stats

from tqdm.notebook import tqdm

In [ ]:
!nvidia-smi

In [ ]:
num_epochs = 25

model_name = "timm/maxvit_tiny_tf_224.in1k"

criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model(model_name, pretrained = True, num_classes = 2).to(device)

In [ ]:
optimizer = timm.optim.create_optimizer_v2(model , opt = "AdamW", lr = 1e-4)
optimizer = timm.optim.Lookahead(optimizer , alpha = 0.5, k = 6)

In [ ]:
device

In [ ]:
def training (num_epochs , train_dataloader , model , criterion , optimizer) :

    model.train()

    for epoch in range(25 , 25 + num_epochs) :

        print(f'epoch = {epoch}')

        losses = 0.0

        for idx , batch in enumerate(tqdm(train_dataloader)) :

            images , labels = batch
            images , labels = images.to(device) , labels.to(device)

            outputs = model(images.to(device))

            loss = criterion(outputs, labels.to(device))
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            losses += loss.item()

            if (idx != 0 and (idx + 1) % 25 == 0 ) :

                print(losses / 25.0)

                losses = 0


        torch.save(model.state_dict(), f'/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/model_checkpoints/maxvit_model_{epoch}.pth')


In [ ]:
training(num_epochs , train_dataloader , model , criterion , optimizer)

In [ ]:
def evaluating (model , validation_dataloader) :

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():

        for idx , batch in enumerate(tqdm(validation_dataloader)) :

            images , labels = batch
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _ , predicted = torch.max(outputs.data, 1)

            total += labels.size(0)

            correct += (predicted == labels.argmax(axis = 1).squeeze()).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')

In [ ]:
evaluating(model , validation_dataloader)